In [0]:
pip install dask[dataframe]

Python interpreter will be restarted.
  Using cached pandas-2.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyiris 2.8.5 requires datadog==0.44.0, but you have datadog 0.36.0 which is incompatible.
pyiris 2.8.5 requires pandas==1.3.5, but you have pandas 2.2.2 which is incompatible.
pyiris 2.8.5 requires pip==22.3.1, but you have pip 21.2.4 which is incompatible.
Python interpreter will be restarted.


In [0]:
rawzone = '/dbfs/mnt/usp/ProjetoIntegrador/rawzone/surftemp-sst/sst.zarr'
bronze = '/dbfs/mnt/usp/ProjetoIntegrador/bronze/surftemp-sst'

In [0]:
def calculate_chunk_sizes(dim_sizes, dtype_size_bytes, max_chunk_size_mb=2):
    import numpy as np
    max_chunk_size_bytes = max_chunk_size_mb * 1024
    max_elements_per_chunk = max_chunk_size_bytes // dtype_size_bytes
    total_elements = np.prod(list(dim_sizes.values()))
    if total_elements * dtype_size_bytes <= max_chunk_size_bytes:
        return {dim: size for dim, size in dim_sizes.items()}
    chunking_factor = total_elements / max_elements_per_chunk
    dim_chunk_sizes = {}
    for dim, size in dim_sizes.items():
        dim_factor = (size / np.prod(list(dim_sizes.values()))) ** (1/len(dim_sizes))
        adjusted_factor = chunking_factor ** dim_factor
        chunk_size = max(1, int(size / adjusted_factor))
        dim_chunk_sizes[dim] = chunk_size
    return dim_chunk_sizes

In [0]:
pip install xarray

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-bfd76fa3-95c2-40cc-b742-2eade7f62f5e/bin/python -m pip install --upgrade pip' command.


In [0]:
import xarray as xr
ds = xr.open_zarr(rawzone)

import xarray as xr
import s3fs
s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root="s3://surftemp-sst/data/sst.zarr", s3=s3, create=False)
ds = xr.open_zarr(store)

In [0]:
del ds['analysed_sst_uncertainty']
del ds['lat_bnds']
del ds['lon_bnds']
del ds['mask']
del ds['sea_ice_fraction']

In [0]:
#corta para os limites do brasil e de seus oceanos em volta
ds = ds.sel(time=slice('2020-01-01', '2022-12-31'), lat=slice(-34, 6), lon=slice(-180, -3.8442823))

In [0]:
dtype_size_bytes = 9

suggested_chunk_sizes = calculate_chunk_sizes(ds.dims.mapping, dtype_size_bytes)

ds = ds.chunk(suggested_chunk_sizes)#{'time': 37, 'lat': 80, 'lon': 90}

#ds = ds.chunk({'time': 37, 'lat': 80, 'lon': 90})


In [0]:
def save_chunk(data_array):
    data_array.to_dataframe().to_csv(f'{temp_dir}/{uuid.uuid4().hex}.csv')

def iterate_chunks(data_chunked, dim):
    chunk_sizes = data_chunked.chunks[dim]
    start = 0
    for size in chunk_sizes:
        slice_ = slice(start, start + size)
        yield data_chunked.isel({dim: slice_})
        start += size

def xarray_to_csv(ds):
    from multiprocessing.pool import ThreadPool
    from tqdm import tqdm
    chunks_array = []
    os.makedirs(f"{temp_dir}", exist_ok=True)
    for chunk_lat in iterate_chunks(ds, 'lat'):
        for chunk_lon in iterate_chunks(chunk_lat, 'lon'):
            for chunk_time in iterate_chunks(chunk_lon, 'time'):
                chunks_array.append(chunk_time)
    with ThreadPool(processes= 20) as p:
        results = tqdm(p.imap(save_chunk, chunks_array), total=len(chunks_array))
        tuple(results)

In [0]:
import os
import uuid

temp_dir = f"{bronze}/temp/"
os.makedirs(temp_dir, exist_ok=True)
chunk_time = xarray_to_csv(ds)

 59%|█████▉    | 2355/4000 [40:57<23:56,  1.14it/s

*** WARNING: max output size exceeded, skipping output. ***

100%|██████████| 4000/4000 [1:10:04<00:00,  1.05s/it]


In [0]:
df = spark.read.csv(f"{bronze[5:]}/temp/*.csv", header=True)

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType, DateType, StructField, StructType, IntegerType

df = (
    df
    .withColumn("time", F.col("time").cast(DateType()))
    .withColumn("lat", F.col("lat").cast(FloatType()))
    .withColumn("lon", F.col("lon").cast(FloatType()))
    .withColumn("analysed_sst", F.col("analysed_sst").cast(FloatType()))
    .withColumn("year", F.col("time").substr(1,4).cast(IntegerType()))
    .withColumn("month", F.col("time").substr(6,2).cast(IntegerType()))
    .withColumn("day", F.col("time").substr(9,2).cast(IntegerType()))
)

In [0]:
df.write.format("avro").mode("overwrite").partitionBy(['year', 'month', 'day']).save(bronze[5:])

In [0]:
spark.read.format("avro").load(bronze[5:]).display()

time,lat,lon,analysed_sst,year,month,day
2020-11-30,-33.975,-121.475,292.31,2020,11,30
2020-11-30,-33.975,-121.425,292.27,2020,11,30
2020-11-30,-33.975,-121.375,292.21,2020,11,30
2020-11-30,-33.975,-121.325,292.13998,2020,11,30
2020-11-30,-33.975,-121.275,292.08,2020,11,30
2020-11-30,-33.975,-121.225,292.05,2020,11,30
2020-11-30,-33.975,-121.175,292.06,2020,11,30
2020-11-30,-33.975,-121.125,292.16,2020,11,30
2020-11-30,-33.975,-121.075,292.28,2020,11,30
2020-11-30,-33.975,-121.025,292.47998,2020,11,30


In [0]:
dbutils.fs.rm(f"{bronze[5:]}/temp/", True)

Out[90]: True